# Debt Market Model

$$
\Delta{t} = t_{k+1} - t_{k}\\
{Q}_{k+1} = {Q}_k + v_1 - v_2 - v_3\\
{D_1}_{k+1} = {D_1}_k + u_1 - u_2 - u_3\\
w_3 = u_3 \cdot \frac{w_2}{u_2}\\
w_1 = [(1+\beta_k)^{\Delta{t}}-1]({D_1}_k+{D_2}_k)\\
{D_2}_{k+1} = {D_2}_k + w_1 - w_2 - w_3\\
{R}_{k+1} = {R}_k + w_2\\
$$

<center>
<img src="./diagrams/debt_dynamics.png"
     alt="Debt dynamics"
     style="width: 60%" />
</center>

<center>
<img src="./diagrams/apt_model.png"
     alt="APT model"
     style="width: 60%" />
</center>

## First phase
* Debt market state -> ETH price changes (exogenous) -> exogenous u,v -> endogenous w -> mutates system state

## Second phase
* APT model, arbitragers act -> u,v activity (to remove diversifiable risk) -> results in change to both debt market and secondary market -> stability controller updates redemption rate and price

## Current Model

1. Historically driven ETH price, locks, and draws (eventually to be driven by APT model)
2. Endogenous liquidation and closing of CDPs
3. Debt market state

# Notes

## Resources
* https://github.com/BlockScience/reflexer/blob/next-steps/next_steps.MD
* https://community-development.makerdao.com/en/learn/vaults/liquidation/

* Close CDPs along debt age distribution around 3 months
* How many CDPs are opened daily?
* How are CDPs closed?
* Assumption: opened vs. topped up CDP e.g. ETH price drops, v1 + u1 increase
  * Rate of change of ETH price, make better assumption about new CDP vs top up
  * Break down daily v1/u1 data into multiple CDPs/top ups based on assumption
  * Extreme events -> indicates top up of existing CDP (one that's fallen below certain liquidation ratio)
  
* Large to small CDP liquidation: 50/50 - 2000/1000 at start of 2019
* 1000 to 2000 active CDPs
* 300% average collat ratio

See [Maker network report](https://www.placeholder.vc/blog/2019/3/1/maker-network-report)

> Towards the end of 2018,collateralization   spiked   to   nearly   400%, perhaps  due  to  heightened  risk-aversion  on the  part  of  CDP  holders,but  has  recently declined  back  to  ~270%,  slightly  under  the system’s average of ~300%.

> As  shown  in  Figure  2A, the average non-empty  CDP  declined  from  above $60Kdaiin  debtat  the  start  of  2018  to  just  over $30Kat  the  start  of  2019.Meanwhile,  the medianCDPby debtgrew from under $500in  debtat  the  start  of  the  year,  reaching around $4Kin   August,before   declining sharply to around$500by early February.

> The  significant delta between  mean and  mediandebts highlights thepower  law distribution acrossCDPs. While small CDPs dominate  by number—with  over  80%  of CDPsdrawingless  than $10K  of  dai—they representjust  over 3%  oftotal debt  in  the system.  On  the  other  end  of  the  spectrum, about 90CDPs (less  than  4%by  number) individually have  more  than $100Kin  dai outstanding,  representing nearly  84%  of  all debt  in  the  system.

> Such concentrationin     debtcan     be problematicfor dai supply.For example, four of the six periodsof dai contractiondiscussed in the previous section were associated with CDPs that  had  over$500K  in  debtbeing liquidated. For  example,  CDP  614 hadover 4.3  million in  debt at  liquidation  on  March 18th, accountingfor much of the contraction in outstanding     dai at     the     time. More dramatically,  the  liquidation  of CDPs  3228 and   3164,on   November   20thand   25threspectively,amounted  to  a  contraction  of over $10.7M in dai, making these two CDPs the primary culprits of thelargest contraction in   daisupplyof2018(i.e.   mid-to-late November as showninFigure 1B).

# Imports

In [ ]:
#%pip install git+https://github.com/danlessa/cadCAD@no_deepcopy

In [ ]:
#%pip show cadCAD

In [ ]:
from shared import *

In [ ]:
import numpy as np
import datetime as dt
import pandas as pd

# Historic MakerDAO Dai debt market activity

In [ ]:
debt_market_df = pd.read_csv('market_model/data/debt_market_df.csv', index_col='date', parse_dates=True)
debt_market_df

In [ ]:
debt_market_df.insert(0, 'seconds_passed', 24 * 3600)
debt_market_df['cumulative_v_1'] = debt_market_df['v_1'].cumsum()

In [ ]:
debt_market_df.plot()

# APT Model Setup

In [ ]:
features = ['beta', 'Q', 'v_1', 'v_2 + v_3', 
                    'D_1', 'u_1', 'u_2', 'u_3', 'u_2 + u_3', 
                    'D_2', 'w_1', 'w_2', 'w_3', 'w_2 + w_3',
                    'D']

features_ml = ['beta', 'Q', 'v_1', 'v_2 + v_3', 'u_1', 'u_2', 'u_3', 'w_1', 'w_2', 'w_3', 'D']

optvars = ['u_1', 'u_2', 'v_1', 'v_2 + v_3']

historical_initial_state = {k: debt_market_df.iloc[0][k] for k in features}
historical_initial_state

In [ ]:
# APT model initial feature vector

# [[1.00000000e+00 5.00000000e-03 2.42566200e+03 2.52666200e+03
#   1.01000000e+02 5.95296083e+05 5.95342027e+05 1.00000000e+01
#   3.59438400e+01 4.59438400e+01 0.00000000e+00 0.00000000e+00
#   6.27647566e-06 2.25600637e-05 2.88365393e-05 5.95296083e+05]]

# 1 1.01760484160946 1 3.86810578185312e-06 365.8127290676168 736.004090277778 0.6756295152422528 1.0004125645956772 0 1.0043275924442232

# {'u_2': 5748489.160776663, 'v_1': 23430.26903522912}

## Root function

In [ ]:
import pickle

model = pickle.load(open('models/pickles/apt_debt_model_2020-11-28.pickle', 'rb'))

# ML debt model root function
def G(x, to_opt, data, constant):
    for i,y in enumerate(x):
        data[:,to_opt[i]] = y
    err = model.predict(data)[0] - constant
    return err

dpres = pickle.load(open('models/pickles/debt_market_OLS_model.pickle', 'rb'))

def G_OLS(x, to_opt, data, constant):
    for i,y in enumerate(x):
        data[:,to_opt[i]] = y
    err = dpres.predict(data)[0] - constant
    #print(f'G_OLS err: {err}')
    return err

ml_data_list = []

# Global minimizer function
def glf(x, to_opt, data, constant, timestep):
    for i,y in enumerate(x):
        #print(x)
        data[:,to_opt[i]] = y
    err = model.predict(data)[0] - constant

    df: pd.DataFrame = pd.read_pickle('exports/ml_data.pickle')
    ml_data = pd.DataFrame([{'x': x, 'to_opt': to_opt, 'data': data, 'constant': constant, 'err': err}])
    ml_data['timestep'] = timestep
    try:
        ml_data['iteration'] = df.iloc[-1]['iteration'] + 1
    except IndexError:
        ml_data['iteration'] = 0
    df.append(ml_data, ignore_index=True).to_pickle('exports/ml_data.pickle')

    #print(err)
    return abs(err)

# Model Configuration

In [ ]:
eth_price = pd.DataFrame(debt_market_df['rho_star'])
eth_p_mean = np.mean(eth_price.to_numpy().flatten())

mar_price = pd.DataFrame(debt_market_df['p'])
mar_p_mean = np.mean(mar_price.to_numpy().flatten())

eth_returns = ((eth_price - eth_price.shift(1))/eth_price.shift(1)).to_numpy().flatten()
eth_gross_returns = (eth_price / eth_price.shift(1)).to_numpy().flatten()

eth_returns_mean = np.mean(eth_returns[1:])

eth_p_mean, eth_returns_mean, mar_p_mean

In [ ]:
#eth_collateral = 2500.0
eth_price = eth_price.iloc[0] #386.71

liquidation_ratio = 1.5 # 150%
liquidation_buffer = 2
#collateral_value = eth_collateral * eth_price
target_price = 1.0
#principal_debt = collateral_value / (target_price * liquidation_ratio * liquidation_buffer)

# print(f'''
# {principal_debt * target_price}
# {collateral_value}
# ''')

In [ ]:
stability_fee = (historical_initial_state['beta'] * 30 / 365) / (30 * 24 * 3600)

In [ ]:
# Create a "genesis" CDP

genesis_cdp_count = 1

cdp_list = []
for i in range(genesis_cdp_count):
    cdp_list.append({
        'open': 1, # True/False == 1/0 for integer/float series
        'time': 0,
        'locked': historical_initial_state['v_1'],
        'drawn': historical_initial_state['u_1'],
        'wiped': historical_initial_state['u_2'],
        'freed': 0.0,
        'w_wiped': historical_initial_state['w_2'],
        'v_bitten': historical_initial_state['v_2 + v_3'],
        'u_bitten': historical_initial_state['u_3'],
        'w_bitten': historical_initial_state['w_3'],
        'dripped': 0.0
    })

cdps = pd.DataFrame(cdp_list)
cdps

In [ ]:
partial_results = pd.DataFrame()
partial_results_file = 'exports/partial_results.pickle'
partial_results.to_pickle(partial_results_file)

ml_data = pd.DataFrame()
ml_data_file = 'exports/ml_data.pickle'
ml_data.to_pickle(ml_data_file)

In [ ]:
initial_state = {
    'events': [],
    'cdps': cdps,
    # Loaded from exogenous parameter
    'eth_price': eth_price.iloc[0], # dollars
    # v
    'eth_collateral': historical_initial_state['Q'] * genesis_cdp_count, # Q
    'eth_locked': historical_initial_state['v_1'] * genesis_cdp_count, # v1
    'eth_freed': 0.0 * genesis_cdp_count, # v2
    'eth_bitten': historical_initial_state['v_2 + v_3'] * genesis_cdp_count, # v3
    'v_1': historical_initial_state['v_1'],
    'v_2': 0.0,
    'v_3': historical_initial_state['v_2 + v_3'],
    # u
    'principal_debt': historical_initial_state['D_1'] * genesis_cdp_count, # D1
    'rai_drawn': historical_initial_state['u_1'] * genesis_cdp_count, # u1 "minted"
    'rai_wiped': historical_initial_state['u_2'] * genesis_cdp_count, # u2
    'rai_bitten': historical_initial_state['u_3'] * genesis_cdp_count, # u3
    'u_1': historical_initial_state['u_1'],
    'u_2': historical_initial_state['u_2'],
    'u_3': historical_initial_state['u_3'],
    # w
    'w_1': historical_initial_state['w_1'],
    'w_2': historical_initial_state['w_2'],
    'w_3': historical_initial_state['w_3'],
    'accrued_interest': historical_initial_state['D_2'] * genesis_cdp_count,
    'stability_fee': stability_fee,
    'market_price': debt_market_df.iloc[0]['p'],
    'target_price': target_price, # dollars == redemption price
    'target_rate': 0 / (30 * 24 * 3600), # per second interest rate (X% per month)
    'p_expected': target_price,
    'p_debt_expected': target_price,
}

initial_state.update(historical_initial_state)

parameters = {
    'debug': [True], # Print debug messages (see APT model)
    'raise_on_assert': [False], # See assert_log() in utils.py
    'test': [
        {
            'enable': False,
            'params': {
                'optimal_values': {
                    'v_1': lambda timestep=0: historical_initial_state['v_1'],
                    'v_2 + v_3': lambda timestep=0: historical_initial_state['v_2 + v_3'],
                    'u_1': lambda timestep=0: historical_initial_state['u_1'],
                    'u_2': lambda timestep=0: historical_initial_state['u_2']
                }
            }
        },
        # {
        #     'enable': True,
        #     'params': {
        #         'optimal_values': {
        #             'v_1': lambda timestep=0: 1000,
        #             'v_2 + v_3': lambda timestep=0: 500,
        #             'u_1': lambda timestep=0: 100,
        #             'u_2': lambda timestep=0: 50
        #         }
        #     }
        # },
        # {
        #     'enable': True,
        #     'params': {
        #         'optimal_values': {
        #             'v_1': lambda timestep=0: 500,
        #             'v_2 + v_3': lambda timestep=0: 1000,
        #             'u_1': lambda timestep=0: 50,
        #             'u_2': lambda timestep=0: 100
        #         }
        #     }
        # }
    ],
    'free_memory_states': [['cdps', 'events']], #'cdps',
    #'eth_market_std': [1],
    #'random_state': [np.random.RandomState(seed=0)],
    'liquidation_ratio': [liquidation_ratio], # %
    'liquidation_buffer': [liquidation_buffer], # multiplier applied to CDP collateral by users
    'stability_fee': [lambda timestep, df=debt_market_df: stability_fee], # df.iloc[timestep].beta / (365 * 24 * 3600), # per second interest rate (1.5% per month)
    'liquidation_penalty': [0], # 0.13 == 13%
    'cdp_top_up_buffer': [2],
    # Average CDP duration == 3 months: https://www.placeholder.vc/blog/2019/3/1/maker-network-report
    # The tuning of this parameter is probably off the average, because we don't have the CDP size distribution matched yet,
    # so although the individual CDPs could have an average debt age of 3 months, the larger CDPs likely had a longer debt age.
    'average_debt_age': [3 * (30 * 24 * 3600)], # delta t (seconds)
    'eth_price': [lambda timestep, df=debt_market_df: df.iloc[timestep].rho_star],
    #'v_1': [lambda state, _state_history, df=debt_market_df: df.iloc[state['timestep']].v_1], # Driven by historical data
    #'u_1': [lambda timestep, df=debt_market_df: df.iloc[timestep].u_1], # Driven by historical data
    'seconds_passed': [lambda timestep, df=debt_market_df: df.iloc[timestep].seconds_passed],
    # 'market_price': [lambda timestep, df=debt_market_df: target_price],
    # APT model
    # **{
    #     'use_APT_ML_model': [False],
    #     'root_function': [G_OLS], # glf, G, G_OLS
    #     'features': [features], # features_ml, features
    # },
    **{
        'use_APT_ML_model': [True],
        'root_function': [glf], # glf, G, G_OLS
        'model': [model],
        'features': [features_ml], # features_ml, features
    },
    'freeze_feature_vector': [False], # Use the same initial state as the feature vector for each timestep
    'optvars': [optvars],
    'bounds': [[(xmin,debt_market_df[optvars].max()[i]) 
        for i,xmin in enumerate(debt_market_df[optvars].min())
        ]],
    'interest_rate': [1.0],
    'eth_p_mean': [eth_p_mean],
    'eth_returns_mean': [eth_returns_mean],
    'mar_p_mean': [mar_p_mean],
    # APT OLS model
    'alpha_0': [0],
    'alpha_1': [1],
    'beta_0': [1.0003953223600617],
    'beta_1': [0.6756295152422528],
    'beta_2': [3.86810578185312e-06],    
    # Controller
    'controller_enabled': [False],
    'kp': [-1.5e-6], #5e-7 #proportional term for the stability controller: units 1/USD
    'ki': [lambda control_period=3600: 0 / control_period], #-1e-7 #integral term for the stability controller: units 1/(USD*seconds)
    'partial_results': [partial_results_file],
}

# parameters = parameters.update({
#     'delta_v1': [lambda state, state_history: delta_v1(state, state_history)],
#     'market_price': [lambda timestep, df=debt_market_df: df.iloc[timestep].p]
# })

# Simulation Execution

In [ ]:
SIMULATION_TIMESTEPS = len(debt_market_df) - 1 # approx. 600
MONTE_CARLO_RUNS = 1

In [ ]:
from models.config_wrapper import ConfigWrapper
import models.system_model_v2 as system_model_v2

system_simulation = ConfigWrapper(system_model_v2, T=range(100), M=parameters, initial_state=initial_state)

In [ ]:
from cadCAD import configs
del configs[:]

system_simulation.append()

(simulation_result, tensor_field, sessions) = run(drop_midsteps=True)

In [ ]:
partial_results: pd.DataFrame = pd.read_pickle(partial_results_file)
partial_results

In [ ]:
partial_results.plot(x='timestamp', y=['eth_collateral', 'principal_debt'])

In [ ]:
# ml_data: pd.DataFrame = pd.read_pickle(ml_data_file)
# ml_data

In [ ]:

# # ml_data.query('timestep == 1').plot(x='iteration', y='err_abs')

# import plotly.express as px
# ml_data: pd.DataFrame = pd.read_pickle(ml_data_file)
# ml_data['err_abs'] = ml_data.err.abs()
# ml_data = ml_data.query('timestep == 1')
# fig = px.line(ml_data, x="iteration", y="err_abs", facet_col="timestep", facet_col_wrap=2, log_y=True)
# fig.update_yaxes(matches=None)
# fig.update_xaxes(matches=None)
# fig.show()

In [ ]:
# Print system events e.g. liquidation assertion errors
simulation_result[simulation_result.events.astype(bool)].events.apply(lambda x: x[0])

# Simulation Analysis

In [ ]:
#simulation_result = pd.concat([simulation_result, debt_market_df.reset_index()], axis=1)

simulation_result = simulation_result.assign(eth_collateral_value = simulation_result.eth_collateral * simulation_result.eth_price)

simulation_result['collateralization_ratio'] = (simulation_result.eth_collateral * simulation_result.eth_price) / (simulation_result.principal_debt * simulation_result.target_price)
#simulation_result['historical_collateralization_ratio'] = (simulation_result.Q * simulation_result.rho_star) / (simulation_result.D_1 * simulation_result.p)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

simulation_result

In [ ]:
from datetime import datetime
simulation_result.to_csv(f'exports/simulation_results/simulation_result-{datetime.now()}.csv')

In [ ]:
#simulation_result = pd.read_csv('simulation_result-2020-12-01 20_29_53.992604.csv')

## Select simulation

In [ ]:
df = simulation_result.query('simulation == 0 and subset == 0')

## Historical ETH price: December 2017 to September 2019

In [ ]:
df.plot(x='timestamp', y=['eth_price'])

In [ ]:
df.plot(x='timestamp', y=['eth_return'])

## Target price / redemption price set to 1 "dollar" for historical comparison

In [ ]:
df.plot(x='timestamp', y=['target_price', 'market_price'])

In [ ]:
df.plot(x='timestamp', y=['p_expected', 'p_debt_expected'])

In [ ]:
df.plot(x='timestamp', y=['target_rate'])

## Historical system ETH collateral vs. model

In [ ]:
df['locked - freed - bitten'] = df['eth_locked'] - df['eth_freed'] - df['eth_bitten']
df.plot(y=['eth_collateral', 'locked - freed - bitten']) #'Q'

## Historical system ETH collateral value vs. model

In [ ]:
df.plot(x='timestamp', y=['eth_collateral_value']) #'C_star'

## Debt market ETH activity

In [ ]:
df.plot(x='timestamp', y=['eth_locked', 'eth_freed', 'eth_bitten'])

In [ ]:
df.plot(x='timestamp', y=['v_1', 'v_2', 'v_3'])

## Debt market principal debt "Rai" activity

In [ ]:
df['drawn - wiped - bitten'] = df['rai_drawn'] - df['rai_wiped'] - df['rai_bitten']
df.plot(x='timestamp', y=['principal_debt', 'drawn - wiped - bitten']) #, 'D_1'

In [ ]:
df.plot(x='timestamp', y=['rai_drawn', 'rai_wiped', 'rai_bitten'])

In [ ]:
df.plot(x='timestamp', y=['u_1', 'u_2', 'u_3'])

## Accrued interest and system revenue (MKR)

In [ ]:
df.plot(x='timestamp', y=['w_1', 'w_2', 'w_3'])

In [ ]:
df.plot(x='timestamp', y=['accrued_interest']) #, 'D_2'

In [ ]:
df.plot(x='timestamp', y=['system_revenue'])

## Historical collateralization ratio vs. model

In [ ]:
df.plot(x='timestamp', y=['collateralization_ratio']) #, 'historical_collateralization_ratio'